In [1]:
# 기본 Library 설정
import numpy as np
import statistics
import random

import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import bytespdate2num
import matplotlib.ticker as ticker
from matplotlib import font_manager, rc
from matplotlib import style
import seaborn as sns

import calendar
import dateutil
from dateutil.parser import parse
import datetime

# Clustering
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram


# 한글 사용하기
import matplotlib.font_manager as fm
from matplotlib import rc
font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

from tqdm import tqdm
import time

# from zeep import Client

from collections import namedtuple
import sqlite3

# 한 번에 matplotlib 그림 띄우기
%matplotlib inline    
%config InlineBackend.figure_format = 'retina'  #%matplotlib 뒤에 써주면 그래프를 더 높은 해상도로 보여줌

# 설정 관련
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [890]:
clustering

,주문자ID,평균 주문금액(median),평균구매주기,전화주문고객,누적결제금액,19이후_누적주문횟수,군집_5,군집_4
0,xigol80,121440.0,37,0.0,3186190,51.0,1,2
1,kyen1311,91620.0,75,0.0,626520,8.0,0,0
2,85643404@n,75820.0,14,0.0,3006060,65.0,1,2
3,ywca897,174080.0,64,0.0,2006080,23.0,3,2
4,pinkbabi100,94100.0,39,0.0,1851270,30.0,3,2
...,...,...,...,...,...,...,...,...
1457,clcc,91270.0,58,0.0,338450,4.0,0,0
1458,pp1664,78335.0,5,0.0,362890,5.0,0,0
1459,90873546@n,76745.0,5,0.0,560040,7.0,0,0
1460,104005455@n,82250.0,1,0.0,374870,9.0,0,0


In [907]:
# Data 전부 불러주기
pck_total_data = pd.read_csv('data/(Main)고객주문통합리스트_세부품목분류완료.csv', encoding='euc-kr')
pck_royal_cst = pd.read_csv('data/(Main)2019이후_3회_이상_주문한_사람들.csv', encoding='euc-kr')
clustering = pd.read_csv('data/clustering완료.csv', encoding='euc-kr')

### 새로운 column들 추가 ###
# Timeseries로 바꿔주고!
pck_total_data['주문일시']= pd.to_datetime(pck_total_data['주문일시'])
pck_total_data['결제일시(입금확인일)']= pd.to_datetime(pck_total_data['결제일시(입금확인일)'])
pck_total_data['발주일']= pd.to_datetime(pck_total_data.발주일)
pck_total_data['배송대기일']= pd.to_datetime(pck_total_data.배송대기일)
pck_total_data['배송시작일']= pd.to_datetime(pck_total_data.배송시작일)
pck_total_data['배송완료일']= pd.to_datetime(pck_total_data.배송완료일)
pck_total_data['주문자 가입일']= pd.to_datetime(pck_total_data['주문자 가입일'])
pck_total_data['가장최근주문']= pd.to_datetime(pck_total_data['가장최근주문'])
del pck_royal_cst['세부분류사이즈']

pck_royal_cst['주문일시']= pd.to_datetime(pck_royal_cst['주문일시'])
pck_royal_cst['결제일시(입금확인일)']= pd.to_datetime(pck_royal_cst['결제일시(입금확인일)'])
pck_royal_cst['발주일']= pd.to_datetime(pck_royal_cst.발주일)
pck_royal_cst['배송대기일']= pd.to_datetime(pck_royal_cst.배송대기일)
pck_royal_cst['배송시작일']= pd.to_datetime(pck_royal_cst.배송시작일)
pck_royal_cst['배송완료일']= pd.to_datetime(pck_royal_cst.배송완료일)
pck_royal_cst['주문자 가입일']= pd.to_datetime(pck_royal_cst['주문자 가입일'])
pck_royal_cst['가장최근주문']= pd.to_datetime(pck_royal_cst['가장최근주문'])

# pck_total_data와 pck_royal_cst에 세부분류사이즈, 총주문량, 주문연도.월 columns 만들어주기
pck_total_data.loc[(pck_total_data['세부분류'].isna()==False)&(pck_total_data['사이즈'].isna()==False), '세부분류사이즈'] = pck_total_data['세부분류']+"_"+pck_total_data['사이즈']
pck_total_data.loc[(pck_total_data['세부분류'].isna()==True)&(pck_total_data['사이즈'].isna()==False), '세부분류사이즈'] = pck_total_data['사이즈']
pck_total_data.loc[(pck_total_data['세부분류'].isna()==False)&(pck_total_data['사이즈'].isna()==True), '세부분류사이즈'] = pck_total_data['세부분류']
pck_total_data.loc[(pck_total_data['세부분류'].isna()==True)&(pck_total_data['사이즈'].isna()==True), '세부분류사이즈'] = None
pck_total_data['세부분류사이즈'] = pck_total_data['세부분류사이즈'].replace("일회용 숟가락_175mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_total_data['세부분류사이즈'] = pck_total_data['세부분류사이즈'].replace("일회용 숟가락_170mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_total_data['세부분류사이즈'] = pck_total_data['세부분류사이즈'].str.strip()    # 띄어쓰기 때문에 다른 품목으로 인식하는 경우 방지하기 위해서 공백 없애기
pck_total_data['총주문량'] = pck_royal_cst['총주문량']
pck_total_data['주문연도.월'] = pck_royal_cst['주문연도.월']
del pck_total_data['주문연도.월']
pck_total_data['주문연도'] = pck_total_data['주문일시'].dt.year
pck_total_data['주문월'] = pck_total_data['주문일시'].dt.month
pck_total_data['주문연월'] = pck_total_data['주문연도'].astype(str) + '.' + pck_total_data['주문월'].astype(str)

pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==False)&(pck_royal_cst['사이즈'].isna()==False), '세부분류사이즈'] = pck_royal_cst['세부분류']+"_"+pck_royal_cst['사이즈']
pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==True)&(pck_royal_cst['사이즈'].isna()==False), '세부분류사이즈'] = pck_royal_cst['사이즈']
pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==False)&(pck_royal_cst['사이즈'].isna()==True), '세부분류사이즈'] = pck_royal_cst['세부분류']
pck_royal_cst.loc[(pck_royal_cst['세부분류'].isna()==True)&(pck_royal_cst['사이즈'].isna()==True), '세부분류사이즈'] = None
pck_royal_cst['세부분류사이즈'] = pck_royal_cst['세부분류사이즈'].replace("일회용 숟가락_175mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_royal_cst['세부분류사이즈'] = pck_royal_cst['세부분류사이즈'].replace("일회용 숟가락_170mm","일회용 숟가락")    # 일회용 숟가락은 사이즈에 상관없이 통일시켜주기
pck_royal_cst['세부분류사이즈'] = pck_royal_cst['세부분류사이즈'].str.strip()    # 띄어쓰기 때문에 다른 품목으로 인식하는 경우 방지하기 위해서 공백 없애기
del pck_royal_cst['주문연도.월']
pck_royal_cst['주문연도'] = pck_royal_cst['주문일시'].dt.year
pck_royal_cst['주문월'] = pck_royal_cst['주문일시'].dt.month
pck_royal_cst['주문연월'] = pck_royal_cst['주문연도'].astype(str) + '.' + pck_royal_cst['주문월'].astype(str)

# 쓰는 column만 [pck]로 추려주기
pck = pck_total_data[['주문번호', '주문자ID', '총 결제금액', '주문연월','세부분류사이즈', '총주문량', 
                    '개수', '수량','세부분류', '주문일시','19년이후 누적주문횟수',
                    '상품번호']]


# 총 결제금액의 중첩을 피하기 위해서 중첩된 주문일시는 빼주기
pck_dropdup = pck.drop_duplicates(['주문일시'])
pck_royal_cst_dropdup = pck_royal_cst.drop_duplicates(['주문일시'])
pck_total_data_dropdup = pck_total_data.drop_duplicates(['주문일시'])

In [908]:
temp = clustering[['주문자ID', '군집_5']]

In [909]:
pck_total_data = pck_total_data.merge(temp, how='left', on='주문자ID')

In [914]:
pck_total_data.to_csv('2017~2019년 고객주문통합리스트.csv', index=False, encoding='euc-kr')

In [911]:
pck_royal_cst = pck_royal_cst.merge(temp, how='left', on='주문자ID')

In [915]:
pck_royal_cst.to_csv('2017~2019년 고객주문통합리스트(1462명).csv', index=False, encoding='euc-kr')

In [916]:
pck_royal_cst['주문자ID'].nunique()

1462

In [873]:
import pandas as pd
import numpy as np
import dateutil
from dateutil.parser import parse
import datetime
from datetime import timedelta

class Customer:
    def __init__(self, data, name):
        self.name = name
        self.receipt = data[data['주문자ID']==self.name].sort_values(by=['주문일시'])
        
    def order_count(self):
        return self.receipt.shape[0]

    def last_order(self):
        df_cst = self.receipt[self.receipt['주문자ID']==self.name]
        last_order = df_cst['주문일시'].iloc[-1]
        print('{}년 {}월 {}일'.format(last_order.year, last_order.month, last_order.day))
        
    # 특정 기간동안 샀던 품목들과 평균구매주기를 반환
    def purchase_product(self, yyyymmdd = pd.Timestamp.now(), month = 3):
        if type(month) != int:
            raise ValueError("month must be a int")
        self.month = month
        self.start_point = yyyymmdd
        if type(yyyymmdd) is not pd.Timestamp:
            self.start_point = datetime.datetime(year=int(yyyymmdd[:4]), month=int(yyyymmdd[4:6]), day=int(yyyymmdd[6:]))
        elif type(yyyymmdd) is pd.Timestamp:
            self.start_point = yyyymmdd
            
        back_point = self.start_point-timedelta(days=self.month*30)
        print('시작일시 :', back_point)
        print('종료일시 :', self.start_point)
        receipt_limited = self.receipt[self.receipt['주문일시']>back_point]
        receipt_limited = receipt_limited[receipt_limited['주문일시']<self.start_point]
        self.receipt_limited = receipt_limited

        product_list = [product for product in list(receipt_limited['세부분류사이즈'].unique()) if type(product) is str]  
        product_list = list(set(product_list))
        
        list_product_average_order_period = []
        list_product_total_using_period = []
        for product in product_list:
            list_collect_product_order_period = []
            df_specific_product = receipt_limited[receipt_limited['세부분류사이즈']==product].drop_duplicates(['주문일시']).sort_values(by='주문일시') # 해당 품목만 있는 데이터프레임 만들어주기
            
            # 먼저 상품구매경험이 2회 이상인 품목들의 구매주기 모아주기
            if (df_specific_product.shape[0] > 1):
                for i in range(df_specific_product.shape[0]-1):
                    product_order_period = df_specific_product['주문일시'].iloc[i+1] - df_specific_product['주문일시'].iloc[i]
                    if product_order_period.days == 0:    # 하루 내 여러 차례 주문한 것들은 한 번 주문한 것으로 간주
                        list_collect_product_order_period.append([product, 0]) # 구매주기를 품목과 같이 append
                    else:
                        list_collect_product_order_period.append([product, product_order_period]) # 구매주기를 품목과 같이 append
            # 상품구매경험이 1회밖에 없는 사람들의 구매주기는 0으로 계산
            else:  
                for i in range(df_specific_product.shape[0]):
                    list_collect_product_order_period.append([product, 0])

            # 구매주기의 평균 구하기
            total_timedelta = datetime.timedelta(days=0, seconds=0)
            for i in list_collect_product_order_period:
                if type(i[1]) is timedelta:
                    total_timedelta = total_timedelta + i[1]
            average_order_period = round(total_timedelta.days/len(list_collect_product_order_period),2)
            list_product_average_order_period.append([product, average_order_period])    # 품목별 평균구매주기 구했음

            # 상품 누적사용일 구하기
            if (df_specific_product.shape[0] > 1):
                product_1st_end_period = (df_specific_product['주문일시'].iloc[-1] - df_specific_product['주문일시'].iloc[0]).days
                list_product_total_using_period.append([product, product_1st_end_period + average_order_period])
            else:
                list_product_total_using_period.append([product, 0])
        
        # 전체 사용주기에서 제품이랑 누적사용일 zip해줘서 dictionary로 만들고
        product = []
        days = []
        for i in list_product_total_using_period:
            product.append(i[0])
            days.append(i[1])
        day_dict = dict(zip(product, days))

        df_pivot = self.receipt_limited.pivot_table(index='주문연월', columns='세부분류사이즈',
                               values='총주문량', aggfunc=np.sum)
        df_pivot = df_pivot.T.fillna(0).reset_index()
        df_pivot['총합'] = df_pivot.sum(axis=1)
        df_pivot['전체사용일'] = df_pivot['세부분류사이즈'].map(day_dict)
        df_pivot['하루당 사용량'] = round((df_pivot['총합'] / df_pivot['전체사용일']),2)
        final_pivot = df_pivot[['세부분류사이즈','총합','전체사용일','하루당 사용량']]
        final_pivot = final_pivot.dropna(axis=0)
        final_pivot['하루당 사용량'] = final_pivot['하루당 사용량'].replace(np.inf, 0)
        final_pivot = final_pivot.sort_values(by=['총합'], ascending=False)
        self.final_pivot = final_pivot
        return final_pivot

    def daily_demand(self, yyyymmdd = pd.Timestamp.now(), month = 3):
        self.yyyymmdd = yyyymmdd
        self.month = month
        final_pivot = self.purchase_product(self.yyyymmdd, self.month)
        baby_data = dict(zip(final_pivot['세부분류사이즈'].values, final_pivot['하루당 사용량'].values))
        baby_data['주문자ID'] = self.name
        baby_df = pd.DataFrame(baby_data, columns=list(baby_data.keys()), index=[0])
        baby_df = baby_df.dropna(axis=0)
        cols = list(baby_df.columns)
        cols = [cols[-1]]+cols[:-1]
        return baby_df[cols]
    
    def membership(self):
        

In [874]:
ex = Customer(pck, 'xigol80')

In [875]:
ex.name

'xigol80'

In [876]:
ex.last_order()

2019년 5월 3일


In [877]:
ex.order_count()

51

In [878]:
ex.purchase_product(yyyymmdd = '20191102', month=3)

시작일시 : 2019-08-04 00:00:00
종료일시 : 2019-11-02 00:00:00


주문연월,세부분류사이즈,총합,전체사용일,하루당 사용량


In [879]:
ex.purchase_product(yyyymmdd = '20181102', month=17)

시작일시 : 2017-06-10 00:00:00
종료일시 : 2018-11-02 00:00:00


주문연월,세부분류사이즈,총합,전체사용일,하루당 사용량
1,다용도컵_200ml,46000.0,468.0,98.29
3,다용도컵_50ml,36000.0,421.0,85.51
4,다용도컵_80ml,24000.0,347.0,69.16
2,다용도컵_270ml,2000.0,0.0,0.00
5,위생장갑,300.0,188.0,1.60
6,일회용 숟가락,100.0,0.0,0.00
0,고무장갑,3.0,178.0,0.02
7,종이호일_1호,2.0,0.0,0.00


In [888]:
ex.purchase_product(yyyymmdd = '20181102', month=5)

시작일시 : 2018-06-05 00:00:00
종료일시 : 2018-11-02 00:00:00


주문연월,세부분류사이즈,총합,전체사용일,하루당 사용량
1,다용도컵_200ml,12000.0,73.0,164.38
3,다용도컵_50ml,12000.0,73.0,164.38
4,다용도컵_80ml,6000.0,0.0,0.00
2,다용도컵_270ml,2000.0,0.0,0.00
5,위생장갑,200.0,119.0,1.68
6,일회용 숟가락,100.0,0.0,0.00
0,고무장갑,1.0,0.0,0.00


In [887]:
ex.receipt_limited[ex.receipt_limited['세부분류사이즈']=='다용도컵_270ml']

,주문번호,주문자ID,총 결제금액,세부분류사이즈,총주문량,개수,수량,세부분류,주문일시,19년이후 누적주문횟수,상품번호,주문연도,주문월,주문연월
2830,20180614-0000021,xigol80,61600,다용도컵_270ml,2000.0,2000.0,1,다용도컵,2018-06-14 03:29:46,51.0,43,2018,6,2018.6


In [883]:
ex.daily_demand(yyyymmdd = '20181102',month = 13)

시작일시 : 2017-10-08 00:00:00
종료일시 : 2018-11-02 00:00:00


,주문자ID,다용도컵_200ml,다용도컵_50ml,다용도컵_80ml,다용도컵_270ml,위생장갑,일회용 숟가락,고무장갑,종이호일_1호
0,xigol80,98.84,87.21,66.67,0.0,1.6,0.0,0.02,0.0


In [881]:
ex.purchase_product(month=17)

시작일시 : 2018-07-10 17:11:04.396675
종료일시 : 2019-12-02 17:11:04.396675


주문연월,세부분류사이즈,총합,전체사용일,하루당 사용량
1,다용도컵_200ml,28000.0,277.0,101.08
2,다용도컵_50ml,24000.0,246.0,97.56
3,다용도컵_80ml,18000.0,231.0,77.92
5,일회용 숟가락,200.0,77.0,2.60
4,위생장갑,150.0,0.0,0.00
0,고무장갑,2.0,144.0,0.01


In [882]:
ex.daily_demand(month = 17)

시작일시 : 2018-07-10 17:11:04.396675
종료일시 : 2019-12-02 17:11:04.396675


,주문자ID,다용도컵_200ml,다용도컵_50ml,다용도컵_80ml,일회용 숟가락,위생장갑,고무장갑
0,xigol80,101.08,97.56,77.92,2.6,0.0,0.01


# 이제 실제 적용해보기!

In [736]:
# 평균구매주기 계산하기
# 최종으로 모든 id를 합칠 mother_df 만들어주기
mother_columns = list(pck['세부분류사이즈'].unique())
mother_columns.insert(0, '주문자ID')
mother_df = pd.DataFrame(columns=mother_columns)

for customer_id in tqdm(list(pck['주문자ID'].unique())):

    # mother_df에 concat해주기
    mother_df = pd.concat([mother_df, baby_df])


  0%|                                                                                         | 0/1462 [00:00<?, ?it/s]


NameError: name 'mother_df' is not defined

# Class 예시

In [1]:
class Salary:
    def __init__(self, pay, bonus):
        self.pay=pay
        self.bonus=bonus
        
    def annual_salary(self):
        return (self.pay*12) + self.bonus
    
class Employee:
    def __init__(self, name, age, salary):
        self.name=name
        self.age=age
        self.obj_salary = salary
        
    def total_salary(self):
        return self.obj_salary.annual_salary()

salary = Salary(15000, 10000)
emp = Employee('max', 25, salary)
print(emp.total_salary())

190000


In [ ]:
    class KMeans(TransformerMixin, ClusterMixin, BaseEstimator):
        def __init__(self, n_clusters=8, init='k-means++', n_init=10,
                     max_iter=300, tol=1e-4, precompute_distances='auto',
                     verbose=0, random_state=None, copy_x=True,
                     n_jobs=None, algorithm='auto'):
            self.n_clusters = n_clusters
            self.init = init
            self.max_iter = max_iter
            self.tol = tol
            self.precompute_distances = precompute_distances
            self.n_init = n_init
            self.verbose = verbose
            self.random_state = random_state
            self.copy_x = copy_x
            self.n_jobs = n_jobs
            self.algorithm = algorithm

        def _check_test_data(self, X):
            X = check_array(X, accept_sparse='csr', dtype=FLOAT_DTYPES)
            n_samples, n_features = X.shape
            expected_n_features = self.cluster_centers_.shape[1]
            if not n_features == expected_n_features:
                raise ValueError("Incorrect number of features. "
                                 "Got %d features, expected %d" % (
                                     n_features, expected_n_features))

            return X

        def fit(self, X, y=None, sample_weight=None):

            random_state = check_random_state(self.random_state)

            n_init = self.n_init
            if n_init <= 0:
                raise ValueError("Invalid number of initializations."
                                 " n_init=%d must be bigger than zero." % n_init)

            if self.max_iter <= 0:
                raise ValueError(
                    'Number of iterations should be a positive number,'
                    ' got %d instead' % self.max_iter
                )

            # avoid forcing order when copy_x=False
            order = "C" if self.copy_x else None
            X = check_array(X, accept_sparse='csr', dtype=[np.float64, np.float32],
                            order=order, copy=self.copy_x)
            # verify that the number of samples given is larger than k
            if _num_samples(X) < self.n_clusters:
                raise ValueError("n_samples=%d should be >= n_clusters=%d" % (
                    _num_samples(X), self.n_clusters))

            tol = _tolerance(X, self.tol)

            # If the distances are precomputed every job will create a matrix of
            # shape (n_clusters, n_samples). To stop KMeans from eating up memory
            # we only activate this if the created matrix is guaranteed to be
            # under 100MB. 12 million entries consume a little under 100MB if they
            # are of type double.
            precompute_distances = self.precompute_distances
            if precompute_distances == 'auto':
                n_samples = X.shape[0]
                precompute_distances = (self.n_clusters * n_samples) < 12e6
            elif isinstance(precompute_distances, bool):
                pass
            else:
                raise ValueError(
                    "precompute_distances should be 'auto' or True/False"
                    ", but a value of %r was passed" %
                    precompute_distances
                )

            # Validate init array
            init = self.init
            if hasattr(init, '__array__'):
                init = check_array(init, dtype=X.dtype.type, copy=True)
                _validate_center_shape(X, self.n_clusters, init)

                if n_init != 1:
                    warnings.warn(
                        'Explicit initial center position passed: '
                        'performing only one init in k-means instead of n_init=%d'
                        % n_init, RuntimeWarning, stacklevel=2)
                    n_init = 1

            # subtract of mean of x for more accurate distance computations
            if not sp.issparse(X):
                X_mean = X.mean(axis=0)
                # The copy was already done above
                X -= X_mean

                if hasattr(init, '__array__'):
                    init -= X_mean

            # precompute squared norms of data points
            x_squared_norms = row_norms(X, squared=True)

            best_labels, best_inertia, best_centers = None, None, None
            algorithm = self.algorithm
            if self.n_clusters == 1:
                # elkan doesn't make sense for a single cluster, full will produce
                # the right result.
                algorithm = "full"
            if algorithm == "auto":
                algorithm = "full" if sp.issparse(X) else 'elkan'
            if algorithm == "full":
                kmeans_single = _kmeans_single_lloyd
            elif algorithm == "elkan":
                kmeans_single = _kmeans_single_elkan
            else:
                raise ValueError("Algorithm must be 'auto', 'full' or 'elkan', got"
                                 " %s" % str(algorithm))

            seeds = random_state.randint(np.iinfo(np.int32).max, size=n_init)
            if effective_n_jobs(self.n_jobs) == 1:
                # For a single thread, less memory is needed if we just store one
                # set of the best results (as opposed to one set per run per
                # thread).
                for seed in seeds:
                    # run a k-means once
                    labels, inertia, centers, n_iter_ = kmeans_single(
                        X, sample_weight, self.n_clusters,
                        max_iter=self.max_iter, init=init, verbose=self.verbose,
                        precompute_distances=precompute_distances, tol=tol,
                        x_squared_norms=x_squared_norms, random_state=seed)
                    # determine if these results are the best so far
                    if best_inertia is None or inertia < best_inertia:
                        best_labels = labels.copy()
                        best_centers = centers.copy()
                        best_inertia = inertia
                        best_n_iter = n_iter_
            else:
                # parallelisation of k-means runs
                results = Parallel(n_jobs=self.n_jobs, verbose=0)(
                    delayed(kmeans_single)(
                        X, sample_weight, self.n_clusters,
                        max_iter=self.max_iter, init=init,
                        verbose=self.verbose, tol=tol,
                        precompute_distances=precompute_distances,
                        x_squared_norms=x_squared_norms,
                        # Change seed to ensure variety
                        random_state=seed
                    )
                    for seed in seeds)
                # Get results with the lowest inertia
                labels, inertia, centers, n_iters = zip(*results)
                best = np.argmin(inertia)
                best_labels = labels[best]
                best_inertia = inertia[best]
                best_centers = centers[best]
                best_n_iter = n_iters[best]

            if not sp.issparse(X):
                if not self.copy_x:
                    X += X_mean
                best_centers += X_mean

            distinct_clusters = len(set(best_labels))
            if distinct_clusters < self.n_clusters:
                warnings.warn(
                    "Number of distinct clusters ({}) found smaller than "
                    "n_clusters ({}). Possibly due to duplicate points "
                    "in X.".format(distinct_clusters, self.n_clusters),
                    ConvergenceWarning, stacklevel=2
                )

            self.cluster_centers_ = best_centers
            self.labels_ = best_labels
            self.inertia_ = best_inertia
            self.n_iter_ = best_n_iter
            return self

        def fit_predict(self, X, y=None, sample_weight=None):

            return self.fit(X, sample_weight=sample_weight).labels_

        def fit_transform(self, X, y=None, sample_weight=None):

            # Currently, this just skips a copy of the data if it is not in
            # np.array or CSR format already.
            # XXX This skips _check_test_data, which may change the dtype;
            # we should refactor the input validation.
            return self.fit(X, sample_weight=sample_weight)._transform(X)

        def transform(self, X):

            check_is_fitted(self)

            X = self._check_test_data(X)
            return self._transform(X)

        def _transform(self, X):
            return euclidean_distances(X, self.cluster_centers_)

        def predict(self, X, sample_weight=None):

            check_is_fitted(self)

            X = self._check_test_data(X)
            x_squared_norms = row_norms(X, squared=True)
            return _labels_inertia(X, sample_weight, x_squared_norms,
                                   self.cluster_centers_)[0]

        def score(self, X, y=None, sample_weight=None):

            check_is_fitted(self)

            X = self._check_test_data(X)
            x_squared_norms = row_norms(X, squared=True)
            return -_labels_inertia(X, sample_weight, x_squared_norms,
                                    self.cluster_centers_)[1]

    def _mini_batch_step(X, sample_weight, x_squared_norms, centers, weight_sums,
                         old_center_buffer, compute_squared_diff,
                         distances, random_reassign=False,
                         random_state=None, reassignment_ratio=.01,
                         verbose=False):

        # Perform label assignment to nearest centers
        nearest_center, inertia = _labels_inertia(X, sample_weight,
                                                  x_squared_norms, centers,
                                                  distances=distances)

        if random_reassign and reassignment_ratio > 0:
            random_state = check_random_state(random_state)
            # Reassign clusters that have very low weight
            to_reassign = weight_sums < reassignment_ratio * weight_sums.max()
            # pick at most .5 * batch_size samples as new centers
            if to_reassign.sum() > .5 * X.shape[0]:
                indices_dont_reassign = \
                    np.argsort(weight_sums)[int(.5 * X.shape[0]):]
                to_reassign[indices_dont_reassign] = False
            n_reassigns = to_reassign.sum()
            if n_reassigns:
                # Pick new clusters amongst observations with uniform probability
                new_centers = random_state.choice(X.shape[0], replace=False,
                                                  size=n_reassigns)
                if verbose:
                    print("[MiniBatchKMeans] Reassigning %i cluster centers."
                          % n_reassigns)

                if sp.issparse(X) and not sp.issparse(centers):
                    assign_rows_csr(
                        X, new_centers.astype(np.intp, copy=False),
                        np.where(to_reassign)[0].astype(np.intp, copy=False),
                        centers)
                else:
                    centers[to_reassign] = X[new_centers]
            # reset counts of reassigned centers, but don't reset them too small
            # to avoid instant reassignment. This is a pretty dirty hack as it
            # also modifies the learning rates.
            weight_sums[to_reassign] = np.min(weight_sums[~to_reassign])

        # implementation for the sparse CSR representation completely written in
        # cython
        if sp.issparse(X):
            return inertia, _k_means._mini_batch_update_csr(
                X, sample_weight, x_squared_norms, centers, weight_sums,
                nearest_center, old_center_buffer, compute_squared_diff)

        # dense variant in mostly numpy (not as memory efficient though)
        k = centers.shape[0]
        squared_diff = 0.0
        for center_idx in range(k):
            # find points from minibatch that are assigned to this center
            center_mask = nearest_center == center_idx
            wsum = sample_weight[center_mask].sum()

            if wsum > 0:
                if compute_squared_diff:
                    old_center_buffer[:] = centers[center_idx]

                # inplace remove previous count scaling
                centers[center_idx] *= weight_sums[center_idx]

                # inplace sum with new points members of this cluster
                centers[center_idx] += \
                    np.sum(X[center_mask] *
                           sample_weight[center_mask, np.newaxis], axis=0)

                # update the count statistics for this center
                weight_sums[center_idx] += wsum

                # inplace rescale to compute mean of all points (old and new)
                # Note: numpy >= 1.10 does not support '/=' for the following
                # expression for a mixture of int and float (see numpy issue #6464)
                centers[center_idx] = centers[center_idx] / weight_sums[center_idx]

                # update the squared diff if necessary
                if compute_squared_diff:
                    diff = centers[center_idx].ravel() - old_center_buffer.ravel()
                    squared_diff += np.dot(diff, diff)

        return inertia, squared_diff

    def _mini_batch_convergence(model, iteration_idx, n_iter, tol,
                                n_samples, centers_squared_diff, batch_inertia,
                                context, verbose=0):
        """Helper function to encapsulate the early stopping logic"""
        # Normalize inertia to be able to compare values when
        # batch_size changes
        batch_inertia /= model.batch_size
        centers_squared_diff /= model.batch_size

        # Compute an Exponentially Weighted Average of the squared
        # diff to monitor the convergence while discarding
        # minibatch-local stochastic variability:
        # https://en.wikipedia.org/wiki/Moving_average
        ewa_diff = context.get('ewa_diff')
        ewa_inertia = context.get('ewa_inertia')
        if ewa_diff is None:
            ewa_diff = centers_squared_diff
            ewa_inertia = batch_inertia
        else:
            alpha = float(model.batch_size) * 2.0 / (n_samples + 1)
            alpha = 1.0 if alpha > 1.0 else alpha
            ewa_diff = ewa_diff * (1 - alpha) + centers_squared_diff * alpha
            ewa_inertia = ewa_inertia * (1 - alpha) + batch_inertia * alpha

        # Log progress to be able to monitor convergence
        if verbose:
            progress_msg = (
                    'Minibatch iteration %d/%d:'
                    ' mean batch inertia: %f, ewa inertia: %f ' % (
                        iteration_idx + 1, n_iter, batch_inertia,
                        ewa_inertia))
            print(progress_msg)

        # Early stopping based on absolute tolerance on squared change of
        # centers position (using EWA smoothing)
        if tol > 0.0 and ewa_diff <= tol:
            if verbose:
                print('Converged (small centers change) at iteration %d/%d'
                      % (iteration_idx + 1, n_iter))
            return True

        # Early stopping heuristic due to lack of improvement on smoothed inertia
        ewa_inertia_min = context.get('ewa_inertia_min')
        no_improvement = context.get('no_improvement', 0)
        if ewa_inertia_min is None or ewa_inertia < ewa_inertia_min:
            no_improvement = 0
            ewa_inertia_min = ewa_inertia
        else:
            no_improvement += 1


# 원본보존

In [ ]:
# 평균구매주기 계산하기
# 최종으로 모든 id를 합칠 mother_df 만들어주기
mother_columns = list(pck['세부분류사이즈'].unique())
mother_columns.insert(0, '주문자ID')
mother_df = pd.DataFrame(columns=mother_columns)

for customer_id in tqdm(list(pck['주문자ID'].unique())):
    # 주문자ID에 따른 temp 데이터프레임 만들어주기
    temp = pck[pck['주문자ID']== customer_id]
    
    # pivot_table 만들어주기
    temp_pivot = temp.pivot_table(index='주문연도.월', columns='세부분류사이즈',
                               values='총주문량', aggfunc=np.sum)
    temp_pivot = temp_pivot.T.fillna(0).reset_index()
    temp_pivot['sum'] = temp_pivot.sum(axis=1)
    
    # 세부품목과 총누적구매량 pivot_table 구하기
    new_pivot = temp_pivot[['세부분류사이즈','sum']]
    
    ## 고객이 최근 1년간 구매한 상품목록
    # product_list 만들어주기
    # 각 고객이 1년동안 주문하지 않았다는 것은 더이상 그 상품을 안 쓰기에 재고를 보유하지 않는다는 것으로 간주
    semi_product_list = [product for product in list(temp['세부분류사이즈'].unique()) if type(product) is str]  
    product_list = []
    for i in semi_product_list:
        df_1 = temp[temp['세부분류사이즈']==i].drop_duplicates(['주문일시']).sort_values(by='주문일시')
        if df_1['주문일시'].iloc[-1] >= datetime.date(2018, 9,1):
            product_list.append(i)

            
   # 각 품목별로 여태까지 주문했던 구매주기 모으기
    product_average_order_period_list = []
    product_total_using_period_list = []
    for product in product_list:
        collect_product_order_period_list = []
        df_specific_product = temp[temp['세부분류사이즈']==product]    # 해당 품목만 있는 데이터프레임 만들어주기
        df_specific_product = df_specific_product.drop_duplicates(['주문일시'])    # 주문일시 겹치는거 없애주기
        df_specific_product = df_specific_product.sort_values(by='주문일시')    # 시간순으로 데이터프레임 정렬하기
        
        # 먼저 상품구매경험이 2회 이상인 사람들의 구매주기 모아주기
        if (df_specific_product.shape[0] > 1):
            for i in range(df_specific_product.shape[0]-1):
                product_order_period = df_specific_product['주문일시'].iloc[i+1] - df_specific_product['주문일시'].iloc[i]
                if product_order_period.days == 0:    # 하루 내 여러 차례 주문한 것들은 한 번 주문한 것으로 간주
                    present_dt = datetime.datetime(year=2019, month=9, day=17)
                    product_order_period = present_dt - df_specific_product['주문일시'].iloc[i]
                    collect_product_order_period_list.append([product, product_order_period]) # 구매주기를 품목과 같이 append
                else:
                    collect_product_order_period_list.append([product, product_order_period]) # 구매주기를 품목과 같이 append
        # 상품구매경험이 1회밖에 없는 사람들의 구매주기는 2019년 9월 17일에서 빼서 계산
        else:  
            for i in range(df_specific_product.shape[0]):
                present_dt = datetime.datetime(year=2019, month=9, day=17)
                product_order_period = present_dt - df_specific_product['주문일시'].iloc[i]
                collect_product_order_period_list.append([product, product_order_period]) # 구매주기를 품목과 같이 append
                
        # 구매주기의 평균 구하기
        total_timedelta = datetime.timedelta(days=0, seconds=0)
        for i in collect_product_order_period_list:
            total_timedelta = total_timedelta + i[1]
        average_order_period = round(total_timedelta.days/len(collect_product_order_period_list),2)
        product_average_order_period_list.append([product, average_order_period])    # 품목별 평균구매주기 구했음
                 
        # 상품 누적사용일 구하기
        if (df_specific_product.shape[0] > 1):
            product_1st_end_period = (df_specific_product['주문일시'].iloc[-1] - df_specific_product['주문일시'].iloc[0]).days
            product_total_using_period_list.append([product, product_1st_end_period + average_order_period])
        else:
            product_1st_end_period = (datetime.datetime(year=2019, month=9, day=17) - df_specific_product['주문일시'].iloc[0]).days
            product_total_using_period_list.append([product, product_1st_end_period + average_order_period])
        
    # 전체 사용주기에서 제품이랑 누적사용일 zip해줘서 dictionary로 만들고
    product = []
    days = []
    for i in product_total_using_period_list:
        product.append(i[0])
        days.append(i[1])
    day_dict = dict(zip(product, days))

    # 마지막으로 final pivot 만들어주기
    new_pivot['전체사용일'] = new_pivot['세부분류사이즈'].map(day_dict)
    new_pivot['하루당 사용량'] = round((new_pivot['sum'] / new_pivot['전체사용일']),2)
    final_pivot = new_pivot[['세부분류사이즈','하루당 사용량']]

    # mother_df에 concat할 baby_data 만들어주기
    baby_data = dict(zip(final_pivot['세부분류사이즈'].values, final_pivot['하루당 사용량'].values))
    baby_data['주문자ID'] = customer_id
    baby_df = pd.DataFrame(baby_data, columns=list(baby_data.keys()), index=[0])
    
    # mother_df에 concat해주기
    mother_df = pd.concat([mother_df, baby_df])
